In [22]:
!pip install streamlit pyjwt bcrypt python-dotenv pyngrok nltk streamlit-option-menu plotly textstat PyPDF2 -q


In [23]:
%%writefile db.py
import sqlite3
import bcrypt
import datetime
import time

DB_NAME = "users.db"


def init_db():
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()

    c.execute("""CREATE TABLE IF NOT EXISTS users (
        id          INTEGER PRIMARY KEY AUTOINCREMENT,
        username    TEXT    NOT NULL,
        email       TEXT    UNIQUE NOT NULL,
        password    BLOB    NOT NULL,
        security_q  TEXT    NOT NULL,
        security_a  BLOB    NOT NULL,
        created_at  TEXT
    )""")

    c.execute("""CREATE TABLE IF NOT EXISTS password_history (
        id       INTEGER PRIMARY KEY AUTOINCREMENT,
        email    TEXT NOT NULL,
        password BLOB NOT NULL,
        set_at   TEXT NOT NULL
    )""")

    c.execute("""CREATE TABLE IF NOT EXISTS login_attempts (
        email        TEXT PRIMARY KEY,
        attempts     INTEGER DEFAULT 0,
        last_attempt REAL
    )""")

    conn.commit()
    conn.close()


def _ts():
    return datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")


def hash_text(t):
    return bcrypt.hashpw(t.encode(), bcrypt.gensalt())


def verify_text(t, h):
    return bcrypt.checkpw(t.encode(), h)


def register_user(username, email, password, security_q, security_a):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    try:
        now        = _ts()
        hashed_pw  = hash_text(password)
        hashed_ans = hash_text(security_a.lower().strip())
        c.execute(
            "INSERT INTO users (username,email,password,security_q,security_a,created_at) VALUES (?,?,?,?,?,?)",
            (username, email, hashed_pw, security_q, hashed_ans, now)
        )
        c.execute(
            "INSERT INTO password_history (email,password,set_at) VALUES (?,?,?)",
            (email, hashed_pw, now)
        )
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


def authenticate_user(email, password):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT username, password FROM users WHERE email=?", (email,))
    row = c.fetchone()
    conn.close()
    if row and verify_text(password, row[1]):
        _reset_attempts(email)
        return row[0]
    _record_failed_attempt(email)
    return None


def check_user_exists(email):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT 1 FROM users WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()
    return result is not None


MAX_ATTEMPTS    = 5
LOCKOUT_SECONDS = 300


def _record_failed_attempt(email):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    now = time.time()
    c.execute(
        "INSERT OR IGNORE INTO login_attempts (email,attempts,last_attempt) VALUES (?,0,?)",
        (email, now)
    )
    c.execute(
        "UPDATE login_attempts SET attempts=attempts+1, last_attempt=? WHERE email=?",
        (now, email)
    )
    conn.commit()
    conn.close()


def _reset_attempts(email):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("UPDATE login_attempts SET attempts=0 WHERE email=?", (email,))
    conn.commit()
    conn.close()


def is_rate_limited(email):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT attempts, last_attempt FROM login_attempts WHERE email=?", (email,))
    row = c.fetchone()
    conn.close()
    if not row:
        return False, 0
    attempts, last = row
    elapsed = time.time() - last
    if attempts >= MAX_ATTEMPTS and elapsed < LOCKOUT_SECONDS:
        return True, int(LOCKOUT_SECONDS - elapsed)
    if elapsed >= LOCKOUT_SECONDS:
        _reset_attempts(email)
    return False, 0


def get_security_question(email):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT security_q FROM users WHERE email=?", (email,))
    row = c.fetchone()
    conn.close()
    return row[0] if row else None


def verify_security_answer(email, answer):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT security_a FROM users WHERE email=?", (email,))
    row = c.fetchone()
    conn.close()
    return row and verify_text(answer.lower().strip(), row[0])


def check_password_reused(email, new_password):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT password FROM password_history WHERE email=?", (email,))
    rows = c.fetchall()
    conn.close()
    for (h,) in rows:
        if verify_text(new_password, h):
            return True
    return False


def check_is_old_password(email, password):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute(
        "SELECT password, set_at FROM password_history WHERE email=? ORDER BY set_at DESC",
        (email,)
    )
    rows = c.fetchall()
    conn.close()
    for h, set_at in rows:
        if verify_text(password, h):
            return set_at
    return None


def update_password(email, new_password):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    now    = _ts()
    hashed = hash_text(new_password)
    c.execute("UPDATE users SET password=? WHERE email=?", (hashed, email))
    c.execute(
        "INSERT INTO password_history (email,password,set_at) VALUES (?,?,?)",
        (email, hashed, now)
    )
    conn.commit()
    conn.close()


def get_all_users():
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("SELECT id, username, email, security_q, created_at FROM users")
    rows = c.fetchall()
    conn.close()
    return rows


def delete_user(email):
    conn = sqlite3.connect(DB_NAME, check_same_thread=False)
    c = conn.cursor()
    c.execute("DELETE FROM users            WHERE email=?", (email,))
    c.execute("DELETE FROM password_history WHERE email=?", (email,))
    c.execute("DELETE FROM login_attempts   WHERE email=?", (email,))
    conn.commit()
    conn.close()



Overwriting db.py


In [24]:
%%writefile readability.py
import textstat


class ReadabilityAnalyzer:
    def __init__(self, text):
        self.text          = text
        self.num_sentences = textstat.sentence_count(text)
        self.num_words     = textstat.lexicon_count(text, removepunct=True)
        self.num_syllables = textstat.syllable_count(text)
        self.complex_words = textstat.difficult_words(text)
        self.char_count    = textstat.char_count(text)

    def get_all_metrics(self):
        return {
            "Flesch Reading Ease":  textstat.flesch_reading_ease(self.text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(self.text),
            "SMOG Index":           textstat.smog_index(self.text),
            "Gunning Fog":          textstat.gunning_fog(self.text),
            "Coleman-Liau":         textstat.coleman_liau_index(self.text),
        }



Overwriting readability.py


In [25]:
%%writefile otp.py
import random, time, smtplib, os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

OTP_EXPIRY_SECONDS = 300
OTP_MAX_ATTEMPTS   = 5
_otp_store = {}

def _generate_otp():
    return f"{random.randint(0, 999999):06d}"

def _store_otp(email, otp):
    _otp_store[email] = {
        "otp": otp,
        "expires_at": time.time() + OTP_EXPIRY_SECONDS,
        "attempts": 0,
    }

def _send_email(to_email, otp):
    sender_email    = os.environ.get("EMAIL_ID", "")
    sender_password = os.environ.get("EMAIL_APP_PASSWORD", "")

    if not sender_email or not sender_password:
        return False, "Gmail credentials not found. Check that EMAIL_ID and EMAIL_APP_PASSWORD are set in Colab Secrets with Notebook access ON."

    subject = "PolicyNav — Your One-Time Password"
    body = f"""Hello,

Your OTP for PolicyNav account recovery is:

        ┌─────────────┐
        │   {otp}    │
        └─────────────┘

Valid for {OTP_EXPIRY_SECONDS // 60} minutes.

— PolicyNav Security Team
"""
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"]    = sender_email
    msg["To"]      = to_email
    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, timeout=10) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, to_email, msg.as_string())
        return True, ""
    except smtplib.SMTPAuthenticationError:
        return False, "Gmail authentication failed. Use a 16-character App Password, not your regular password."
    except Exception as e:
        return False, f"Error: {e}"

def send_otp(email):
    otp = _generate_otp()
    _store_otp(email, otp)
    ok, err = _send_email(email, otp)
    if not ok:
        _otp_store.pop(email, None)
        return False, err
    return True, ""

def verify_otp(email, entered):
    record = _otp_store.get(email)
    if not record:                              return False, "not_found"
    if time.time() > record["expires_at"]:
        _otp_store.pop(email, None);           return False, "expired"
    if record["attempts"] >= OTP_MAX_ATTEMPTS:
        _otp_store.pop(email, None);           return False, "too_many_attempts"
    if entered.strip() != record["otp"]:
        record["attempts"] += 1;               return False, "invalid"
    _otp_store.pop(email, None)
    return True, ""

def invalidate_otp(email):    _otp_store.pop(email, None)
def otp_pending(email):
    r = _otp_store.get(email)
    if not r: return False
    if time.time() > r["expires_at"]: _otp_store.pop(email, None); return False
    return True
def seconds_until_expiry(email):
    r = _otp_store.get(email)
    if not r: return 0
    return max(0, int(r["expires_at"] - time.time()))

Overwriting otp.py


In [26]:
%%writefile app.py
import streamlit as st
import re
import jwt
import datetime
import time
import db
import otp as otp_module
import readability as rl
import plotly.graph_objects as go
import PyPDF2

# ── Config ─────────────────────────────────────────────────────────
def _get_secret(key, fallback=""):
    try:
        from google.colab import userdata
        val = userdata.get(key)
        if val:
            return val
    except Exception:
        pass
    import os
    return os.getenv(key, fallback)

SECRET_KEY  = _get_secret("JWT_SECRET_KEY", "policynav_secret_key_2025")
ADMIN_EMAIL = _get_secret("ADMIN_EMAIL_ID", "admin@policynav.com")

st.set_page_config(
    page_title="PolicyNav",
    page_icon="◈",
    layout="wide",
    initial_sidebar_state="expanded"
)

def load_css():
    st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Playfair+Display:ital,wght@0,700;1,400&family=DM+Sans:wght@300;400;500&display=swap');

    .stApp { background-color: #f4f1ec; font-family: 'DM Sans', sans-serif; }
    .block-container { padding-top: 0 !important; padding-bottom: 0 !important; max-width: 100% !important; }
    #MainMenu, footer, header, [data-testid="stToolbar"], [data-testid="stDecoration"] { visibility: hidden; }

    .logo { position: fixed; top: 24px; left: 28px; display: flex; align-items: center; gap: 9px; z-index: 999; }
    .logo-icon { width: 40px; height: 40px; background: #1a1a1a; border-radius: 10px; display: flex; align-items: center; justify-content: center; font-size: 17px; color: white; }
    .logo-name { font-size: 14px; font-weight: 500; color: #1a1a1a; letter-spacing: -0.2px; }
    .logo-name span { font-weight: 300; color: #9a948b; }

    .hero { display: flex; flex-direction: column; align-items: center; justify-content: center; min-height: 44vh; text-align: center; padding: 60px 24px 24px; }
    .hero-eye { font-size: 10.5px; font-weight: 500; letter-spacing: 3.5px; color: #9a948b; text-transform: uppercase; margin-bottom: 22px; }
    .hero-h1 { font-family: 'Playfair Display', serif; font-weight: 700; font-size: clamp(50px,6.5vw,78px); color: #1a1a1a; line-height: 1.06; margin-bottom: 18px; }
    .hero-h1 em { font-style: italic; font-weight: 400; color: #7c7c7c; }
    .hero-sub { font-size: 15px; color: #9a948b; font-weight: 300; margin-bottom: 8px; }

    .fh { text-align: center; padding: 80px 0 0; }
    .fh-eye { font-size: 10.5px; letter-spacing: 3.5px; color: #9a948b; text-transform: uppercase; margin-bottom: 10px; }
    .fh-title { font-family: 'Playfair Display', serif; font-size: 38px; font-weight: 700; color: #1a1a1a; margin-bottom: 8px; }
    .fh-title em { font-style: italic; font-weight: 400; color: #7c7c7c; }
    .fh-sub { font-size: 14px; color: #9a948b; font-weight: 300; margin-bottom: 0; }
    .fh-rule { width: 36px; height: 1px; background: #d4cfc8; margin: 28px auto 32px; }

    .stTextInput > label { font-size: 11px !important; font-weight: 500 !important; letter-spacing: 1.8px !important; text-transform: uppercase !important; color: #7c7c7c !important; margin-bottom: 5px !important; }
    .stTextInput > div > div > input { background: #ffffff !important; border: 1px solid #e0dbd3 !important; border-radius: 10px !important; padding: 14px 16px !important; font-family: 'DM Sans', sans-serif !important; font-size: 15px !important; color: #1a1a1a !important; box-shadow: 0 1px 3px rgba(0,0,0,0.04) !important; }
    .stTextInput > div > div > input:focus { border-color: #1a1a1a !important; box-shadow: 0 0 0 3px rgba(26,26,26,0.07) !important; }
    input { color: #1a1a1a !important; -webkit-text-fill-color: #1a1a1a !important; }
    input::placeholder { color: #b8b2aa !important; font-weight: 300 !important; }
    .stSelectbox > label { font-size: 11px !important; font-weight: 500 !important; letter-spacing: 1.8px !important; text-transform: uppercase !important; color: #7c7c7c !important; }
    div[data-baseweb="select"] > div { background: #ffffff !important; border: 1px solid #e0dbd3 !important; border-radius: 10px !important; color: #1a1a1a !important; }

    .stButton > button { font-family: 'DM Sans', sans-serif !important; font-size: 14px !important; font-weight: 500 !important; background-color: #1a1a1a !important; color: #ffffff !important; border: none !important; border-radius: 100px !important; padding: 13px 28px !important; width: 100% !important; transition: all 0.18s ease !important; }
    .stButton > button:hover { background-color: #2d2d2d !important; transform: translateY(-1px) !important; box-shadow: 0 5px 18px rgba(0,0,0,0.16) !important; }

    .ghost .stButton > button { background-color: transparent !important; color: #7c7c7c !important; border: 1px solid #d4cfc8 !important; font-size: 13px !important; }
    .ghost .stButton > button p, .ghost .stButton > button span { color: #7c7c7c !important; }
    .ghost .stButton > button:hover { background-color: #ede9e2 !important; color: #1a1a1a !important; box-shadow: none !important; }
    .ghost .stButton > button:hover p, .ghost .stButton > button:hover span { color: #1a1a1a !important; }

    .stSuccess > div { background: #f2faf5 !important; border: 1px solid #a7d9b8 !important; border-radius: 10px !important; color: #145a32 !important; }
    .stError   > div { background: #fff6f6 !important; border: 1px solid #f5bcbc !important; border-radius: 10px !important; color: #7b1a1a !important; }
    .stInfo    > div { background: #f5f3ff !important; border: 1px solid #c4b9f5 !important; border-radius: 10px !important; }
    .stWarning > div { background: #fffbf0 !important; border: 1px solid #f5dfa0 !important; border-radius: 10px !important; }

    .strength-bar { height: 5px; border-radius: 3px; margin-top: 8px; }
    .strength-label { font-size: 11px; letter-spacing: 1.5px; text-transform: uppercase; margin-top: 4px; font-weight: 500; }

    .dash-card { background: #ffffff; border-radius: 18px; padding: 44px 40px; box-shadow: 0 2px 24px rgba(0,0,0,0.06); text-align: center; margin-bottom: 24px; }
    .dash-card h2 { font-family: 'Playfair Display', serif; font-size: 30px; color: #1a1a1a; margin-bottom: 6px; }
    .dash-card h2 em { font-style: italic; font-weight: 400; color: #7c7c7c; }
    .dash-card p { font-size: 13px; color: #9a948b; margin: 0; }

    section[data-testid="stSidebar"] { background-color: #1a1a1a !important; border-right: 1px solid #2d2d2d !important; }
    section[data-testid="stSidebar"] .stButton > button { background-color: #2d2d2d !important; border: 1px solid #3d3d3d !important; color: #e8e3dc !important; border-radius: 8px !important; }
    section[data-testid="stSidebar"] .stButton > button:hover { background-color: #3d3d3d !important; }
    section[data-testid="stSidebar"] .stRadio > label { color: #9a948b !important; }
    section[data-testid="stSidebar"] .stRadio div[role="radiogroup"] label p { color: #e8e3dc !important; }
    section[data-testid="stSidebar"] [data-testid="stMarkdownContainer"] p { color: #e8e3dc !important; }
    section[data-testid="stSidebar"] [data-testid="stMarkdownContainer"] div { color: #e8e3dc !important; }

    .stApp p, .stApp label, .stApp h1, .stApp h2, .stApp h3, .stApp h4, .stApp h5 { color: #1a1a1a; }
    .stButton > button, .stButton > button p, .stButton > button span, .stButton > button div { color: #ffffff !important; }
    .logo-name { color: #1a1a1a !important; }
    .logo-name span { color: #9a948b !important; }
    [data-testid="stMetricLabel"] p { color: #7c7c7c !important; }
    [data-testid="stMetricValue"]   { color: #1a1a1a !important; }
    .stTabs [data-baseweb="tab"] p  { color: #1a1a1a !important; font-weight: 500; }
    .stTabs [data-baseweb="tab-list"] { background: transparent !important; }
    .streamlit-expanderHeader p     { color: #1a1a1a !important; font-weight: 500; }
    .streamlit-expanderContent p    { color: #5a5a5a !important; }
    details summary p               { color: #1a1a1a !important; }
    details div p                   { color: #5a5a5a !important; }
    [data-testid="stExpander"] summary span { color: #1a1a1a !important; }
    [data-testid="stExpander"] p    { color: #5a5a5a !important; }
    [data-testid="stFileUploader"] label, [data-testid="stFileUploader"] label p,
    [data-testid="stFileUploader"] span, [data-testid="stFileUploader"] small,
    [data-testid="stFileUploader"] p { color: #1a1a1a !important; }
    [data-testid="stFileUploadDropzone"] span { color: #ffffff !important; }
    [data-testid="stFileUploadDropzone"] p    { color: #ffffff !important; }
    [data-testid="stFileUploadDropzone"] small { color: #aaaaaa !important; }
    [data-testid="stFileUploadDropzone"] button { border: 1px solid #ffffff !important; border-radius: 6px !important; color: #ffffff !important; }
    [data-testid="stFileUploaderFile"] span  { color: #1a1a1a !important; }
    [data-testid="stFileUploaderFile"] p     { color: #1a1a1a !important; }
    [data-testid="stFileUploaderFile"] small { color: #7c7c7c !important; }
    [data-testid="stFileUploader"] [data-testid="stFileUploaderFile"] a { color: #1a1a1a !important; text-decoration: none !important; }
    .stTextArea label p  { color: #7c7c7c !important; }
    .stTextArea textarea { color: #1a1a1a !important; background: #ffffff !important; }
    .stRadio label p     { color: #1a1a1a !important; }
    div[data-baseweb="select"] span { color: #1a1a1a !important; }
    .stSpinner p         { color: #1a1a1a !important; }
    .stInfo    > div p   { color: #1a1a1a !important; }
    .stSuccess > div p   { color: #145a32 !important; }
    .stError   > div p   { color: #7b1a1a !important; }
    .stWarning > div p   { color: #7a5200 !important; }
    section[data-testid="stSidebar"] p, section[data-testid="stSidebar"] span,
    section[data-testid="stSidebar"] label, section[data-testid="stSidebar"] div { color: #e8e3dc !important; }
    section[data-testid="stSidebar"] .stRadio label p { color: #e8e3dc !important; }

    .level-badge { padding: 16px 20px; border-radius: 12px; font-family: 'Playfair Display', serif; font-size: 22px; font-weight: 700; text-align: center; margin-bottom: 20px; }
    .otp-banner { background: #f5f3ff; border: 1px solid #c4b9f5; border-radius: 12px; padding: 16px 20px; margin: 12px 0; font-size: 13px; color: #3d2d8a; }
    .sp8  { height: 8px;  }
    .sp16 { height: 16px; }
    .sp24 { height: 24px; }
    </style>
    """, unsafe_allow_html=True)


def logo():
    st.markdown("""
    <div class="logo">
      <div class="logo-icon">◈</div>
      <div class="logo-name">Policy<span>Nav</span></div>
    </div>""", unsafe_allow_html=True)

def sp(cls="sp8"):
    st.markdown(f"<div class='{cls}'></div>", unsafe_allow_html=True)

def ghost_back(label, dest, key):
    st.markdown("<div class='ghost'>", unsafe_allow_html=True)
    if st.button(label, key=key):
        st.session_state.page = dest
        st.rerun()
    st.markdown("</div>", unsafe_allow_html=True)

def validate_email(e):
    return re.match(r"^[a-zA-Z0-9._%+\-]+@[a-zA-Z0-9.\-]+\.[a-zA-Z]{2,}$", e)

def check_password_strength(password):
    if not password:
        return "-", "#d4cfc8", 0, ""
    if " " in password:
        return "Weak", "#e74c3c", 10, "No spaces allowed"
    if not password.isalnum():
        return "Weak", "#e74c3c", 10, "Only letters and digits allowed (no special characters)"
    if len(password) <= 8:
        return "Weak", "#e74c3c", 25, f"Too short ({len(password)}/8+) — must be MORE than 8 characters"
    has_letter = bool(re.search(r"[a-zA-Z]", password))
    has_digit  = bool(re.search(r"\d", password))
    if not has_letter or not has_digit:
        return "Medium", "#f39c12", 50, "Mix letters and digits (e.g. Pass1234)"
    if len(password) >= 12:
        return "Strong", "#27ae60", 100, "Great password!"
    return "Medium", "#f39c12", 70, "Good — longer passwords are stronger"

def show_strength_meter(password):
    level, color, pct, tip = check_password_strength(password)
    if password:
        st.markdown(f"""
        <div class="strength-bar" style="width:{pct}%; background:{color};"></div>
        <div class="strength-label" style="color:{color};">{level}{' — ' + tip if tip else ''}</div>
        """, unsafe_allow_html=True)

def create_token(email, username):
    return jwt.encode({
        "email": email, "username": username,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(minutes=60)
    }, SECRET_KEY, algorithm="HS256")

def verify_token(token):
    try:    return jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
    except: return None

def get_relative_time(date_str):
    if not date_str:
        return "some time ago"
    try:
        past = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        diff = datetime.datetime.utcnow() - past
        s    = diff.seconds
        if diff.days > 365:  return f"{diff.days // 365} year(s) ago"
        if diff.days > 30:   return f"{diff.days // 30} month(s) ago"
        if diff.days > 0:    return f"{diff.days} day(s) ago"
        if s > 3600:         return f"{s // 3600} hour(s) ago"
        if s > 60:           return f"{s // 60} minute(s) ago"
        return "just now"
    except:
        return date_str

def create_gauge(value, title, min_val=0, max_val=100, color="#1a1a1a"):
    fig = go.Figure(go.Indicator(
        mode  = "gauge+number",
        value = value,
        title = {'text': title, 'font': {'size': 13, 'color': color}},
        number= {'font': {'size': 20, 'color': color}},
        gauge = {
            'axis':        {'range': [min_val, max_val], 'tickwidth': 1},
            'bar':         {'color': color},
            'bgcolor':     "#f4f1ec",
            'borderwidth': 2,
            'bordercolor': "#d4cfc8",
            'steps':       [{'range': [min_val, max_val], 'color': "#f9f7f3"}],
        }
    ))
    fig.update_layout(
        paper_bgcolor="#ffffff",
        font={'family': "DM Sans"},
        height=230,
        margin=dict(l=10, r=10, t=40, b=10)
    )
    return fig

_defaults = {
    "page":            "home",
    "reset_email":     None,
    "question_shown":  False,
    "answer_verified": False,
    "token":           None,
    "username":        None,
    "otp_sent":        False,
    "otp_verified":    False,
}
for k, v in _defaults.items():
    if k not in st.session_state:
        st.session_state[k] = v


def main():
    load_css()
    db.init_db()
    logo()

    # HOME
    if st.session_state.page == "home":
        st.markdown("""
        <div class="hero">
          <div class="hero-eye">Secure · Intelligent · Editorial</div>
          <div class="hero-h1">Navigate with<br><em>confidence.</em></div>
          <div class="hero-sub">Secure access · Readability analytics · Policy insights.</div>
        </div>""", unsafe_allow_html=True)
        _, col, _ = st.columns([1.2, 1, 1.2])
        with col:
            if st.button("Sign in",         key="h_li"): st.session_state.page = "login";  st.rerun()
            sp()
            if st.button("Create account",  key="h_su"): st.session_state.page = "signup"; st.rerun()
            sp()
            if st.button("Forgot password", key="h_fp"): st.session_state.page = "forgot"; st.rerun()

    # LOGIN
    elif st.session_state.page == "login":
        st.markdown("""
        <div class="fh">
          <div class="fh-eye">Welcome back</div>
          <div class="fh-title">Sign <em>in.</em></div>
          <div class="fh-sub">Enter your credentials to continue.</div>
          <div class="fh-rule"></div>
        </div>""", unsafe_allow_html=True)
        _, col, _ = st.columns([1, 1.6, 1])
        with col:
            email = st.text_input("Email",    placeholder="you@example.com", key="li_em")
            pw    = st.text_input("Password", placeholder="••••••••", type="password", key="li_pw")
            sp("sp16")
            if st.button("Sign in →", key="li_btn"):
                if not email or not pw:
                    st.error("Please fill in all fields.")
                else:
                    locked, wait = db.is_rate_limited(email)
                    if locked:
                        st.error(f"⛔ Too many failed attempts. Try again in {wait}s.")
                    else:
                        username = db.authenticate_user(email, pw)
                        if username:
                            st.session_state.token    = create_token(email, username)
                            st.session_state.username = username
                            st.session_state.page     = "dashboard"
                            st.rerun()
                        else:
                            st.error("Invalid email or password.")
                            old_dt = db.check_is_old_password(email, pw)
                            if old_dt:
                                st.warning(f"⚠️ This matches a previous password used {get_relative_time(old_dt)}. Please use your latest password.")
            sp()
            ghost_back("← Back", "home", "li_back")

    # SIGN UP
    elif st.session_state.page == "signup":
        st.markdown("""
        <div class="fh">
          <div class="fh-eye">Get started</div>
          <div class="fh-title">Create an <em>account.</em></div>
          <div class="fh-sub">Join PolicyNav in seconds.</div>
          <div class="fh-rule"></div>
        </div>""", unsafe_allow_html=True)
        _, col, _ = st.columns([1, 1.6, 1])
        with col:
            username   = st.text_input("Username",         placeholder="Your name",        key="su_un")
            email      = st.text_input("Email",            placeholder="you@example.com",  key="su_em")
            pw         = st.text_input("Password",         placeholder="Min 8 characters", type="password", key="su_pw")
            show_strength_meter(pw)
            sp("sp8")
            confirm    = st.text_input("Confirm Password", placeholder="Same as above",    type="password", key="su_cp")
            security_q = st.selectbox("Security Question", [
                "What is your pet name?",
                "What is your favourite colour?",
                "What is your favourite book?",
                "What is your favourite movie?",
                "Who is your favourite teacher?"
            ], key="su_sq")
            security_a = st.text_input("Security Answer",  placeholder="No spaces",        key="su_sa")
            sp("sp16")
            if st.button("Create account →", key="su_btn"):
                if not all([username, email, pw, confirm, security_a]):
                    st.error("All fields are required.")
                elif not validate_email(email):
                    st.error("Invalid email format.")
                elif " " in pw or " " in security_a:
                    st.error("No spaces allowed in password or security answer.")
                elif pw != confirm:
                    st.error("Passwords do not match.")
                else:
                    level, _, _, tip = check_password_strength(pw)
                    if level == "Weak":
                        st.error(f"Password too weak: {tip}")
                    elif db.register_user(username, email, pw, security_q, security_a):
                        st.success("✅ Account created — please sign in.")
                    else:
                        st.error("Email already registered.")
            sp()
            ghost_back("← Back", "home", "su_back")

    # FORGOT PASSWORD
    elif st.session_state.page == "forgot":
        st.markdown("""
        <div class="fh">
          <div class="fh-eye">Account recovery</div>
          <div class="fh-title">Reset your <em>password.</em></div>
          <div class="fh-sub">OTP verification · Security question · New password.</div>
          <div class="fh-rule"></div>
        </div>""", unsafe_allow_html=True)

        _, col, _ = st.columns([1, 1.6, 1])
        with col:

            # STEP 1: Email
            email = st.text_input(
                "Registered Email",
                placeholder="you@example.com",
                key="fp_em",
                disabled=st.session_state.otp_sent,
            )
            if not st.session_state.otp_sent:
                if st.button("Send OTP →", key="fp_send"):
                    if not email:
                        st.error("Please enter your email.")
                    elif not db.check_user_exists(email):
                        st.error("Email not found.")
                    else:
                        with st.spinner("Sending OTP to your email…"):
                            ok, err = otp_module.send_otp(email)
                        if ok:
                            st.session_state.reset_email = email
                            st.session_state.otp_sent    = True
                            st.rerun()
                        else:
                            st.error(f"Could not send OTP: {err}")

            # STEP 2: OTP verification — LIVE COUNTDOWN
            if st.session_state.otp_sent and not st.session_state.otp_verified:
                timer_placeholder = st.empty()
                secs = otp_module.seconds_until_expiry(st.session_state.reset_email)
                mins, sec = divmod(secs, 60)
                time_color = "#e74c3c" if secs < 60 else "#3d2d8a"
                timer_placeholder.markdown(f"""
                <div class="otp-banner">
                  📧 OTP sent to <strong>{st.session_state.reset_email}</strong> —
                  expires in <strong style="color:{time_color}; font-size:16px; font-family:monospace;">{mins:02d}:{sec:02d}</strong>
                </div>
                """, unsafe_allow_html=True)

                entered_otp = st.text_input(
                    "One-Time Password (6 digits)",
                    placeholder="e.g. 482901",
                    max_chars=6,
                    key="fp_otp",
                )
                c1, c2 = st.columns(2)
                with c1:
                    if st.button("Verify OTP →", key="fp_ver_otp"):
                        ok, reason = otp_module.verify_otp(st.session_state.reset_email, entered_otp)
                        if ok:
                            st.session_state.otp_verified   = True
                            st.session_state.question_shown = True
                            st.rerun()
                        elif reason == "expired":
                            st.error("⏰ OTP expired. Please request a new one.")
                            st.session_state.otp_sent = False
                            st.rerun()
                        elif reason == "too_many_attempts":
                            st.error("⛔ Too many wrong attempts. Please request a new OTP.")
                            st.session_state.otp_sent = False
                            st.rerun()
                        else:
                            st.error("❌ Incorrect OTP. Please try again.")
                with c2:
                    st.markdown("<div class='ghost'>", unsafe_allow_html=True)
                    if st.button("Resend OTP", key="fp_resend"):
                        with st.spinner("Resending…"):
                            ok, err = otp_module.send_otp(st.session_state.reset_email)
                        if ok:
                            st.success("✅ New OTP sent!")
                            st.rerun()
                        else:
                            st.error(f"Could not resend: {err}")
                    st.markdown("</div>", unsafe_allow_html=True)

                # Live tick — updates timer every second
                time.sleep(1)
                st.rerun()

            # STEP 3: Security question
            if st.session_state.otp_verified and st.session_state.question_shown and not st.session_state.answer_verified:
                sp()
                q = db.get_security_question(st.session_state.reset_email)
                st.success(f"✅ OTP verified! Security question: **{q}**")
                answer = st.text_input("Security Answer", placeholder="Your answer", key="fp_ans")
                if st.button("Verify answer →", key="fp_ver_ans"):
                    if " " in answer:
                        st.error("No spaces allowed.")
                    elif db.verify_security_answer(st.session_state.reset_email, answer):
                        st.session_state.answer_verified = True
                        st.rerun()
                    else:
                        st.error("Incorrect answer.")

            # STEP 4: New password
            if st.session_state.answer_verified:
                sp()
                new_pw  = st.text_input("New Password",     placeholder="Min 8 chars",   type="password", key="fp_np")
                show_strength_meter(new_pw)
                sp("sp8")
                conf_pw = st.text_input("Confirm Password", placeholder="Same as above", type="password", key="fp_cp")
                if st.button("Reset password →", key="fp_rst"):
                    if new_pw != conf_pw:
                        st.error("Passwords do not match.")
                    elif " " in new_pw:
                        st.error("No spaces allowed.")
                    elif db.check_password_reused(st.session_state.reset_email, new_pw):
                        st.error("⚠️ You cannot reuse a previous password.")
                    else:
                        level, _, _, tip = check_password_strength(new_pw)
                        if level == "Weak":
                            st.error(f"Password too weak: {tip}")
                        else:
                            db.update_password(st.session_state.reset_email, new_pw)
                            st.success("✅ Password reset — please sign in.")
                            for k in ["question_shown","answer_verified","otp_sent","otp_verified"]:
                                st.session_state[k] = False
                            st.session_state.reset_email = None

            sp()
            if st.button("← Back", key="fp_back"):
                if st.session_state.get("reset_email"):
                    otp_module.invalidate_otp(st.session_state.reset_email)
                for k in ["otp_sent","otp_verified","question_shown","answer_verified"]:
                    st.session_state[k] = False
                st.session_state.reset_email = None
                st.session_state.page = "home"
                st.rerun()

    # DASHBOARD
    elif st.session_state.page == "dashboard":
        payload = verify_token(st.session_state.get("token"))
        if not payload:
            st.session_state.page = "login"
            st.rerun()
            return

        uname  = payload["username"]
        uemail = payload["email"]

        with st.sidebar:
            st.markdown(f"""
            <div style='padding:24px 0 8px; text-align:center;'>
              <div style='font-size:28px;'>◈</div>
              <div style='font-family:DM Sans; font-size:16px; font-weight:500; margin-top:6px; color:#e8e3dc;'>PolicyNav</div>
              <div style='font-size:12px; opacity:0.5; margin-top:4px; color:#9a948b;'>{uemail}</div>
            </div>
            <hr style='border-color:#2d2d2d; margin:12px 0;'/>
            """, unsafe_allow_html=True)
            nav = st.radio("Navigation", ["🏠  Home", "📖  Readability", "🛡️  Admin"],
                           label_visibility="collapsed", key="nav_radio")
            st.markdown("<hr style='border-color:#2d2d2d; margin:12px 0;'/>", unsafe_allow_html=True)
            if st.button("🔓  Sign out", key="sb_logout"):
                for k in ["token", "username"]:
                    st.session_state.pop(k, None)
                st.session_state.page = "home"
                st.rerun()

        if nav == "🏠  Home":
            _, col, _ = st.columns([1, 1.6, 1])
            with col:
                sp("sp24")
                st.markdown(f"""
                <div class="dash-card">
                  <h2>Welcome back,<br><em>{uname}</em></h2>
                  <p>{uemail}</p>
                </div>""", unsafe_allow_html=True)
                st.info("✅ You are securely authenticated. Use the sidebar to explore features.")

        elif nav == "📖  Readability":
            st.markdown("""
            <div style='padding:40px 0 24px;'>
              <div style='font-size:10.5px; letter-spacing:3.5px; color:#9a948b; text-transform:uppercase; margin-bottom:10px;'>Text Analytics</div>
              <div style='font-family:Playfair Display,serif; font-size:36px; font-weight:700; color:#1a1a1a;'>
                Readability <em style='font-style:italic; font-weight:400; color:#7c7c7c;'>Analyzer</em>
              </div>
            </div>
            """, unsafe_allow_html=True)
            st.markdown("<style>.stTabs [data-baseweb='tab'] p { color: #1a1a1a !important; font-weight:500; } .stTabs [aria-selected='true'] p { color: #1a1a1a !important; }</style>", unsafe_allow_html=True)
            tab1, tab2 = st.tabs(["✍️ Paste Text", "📂 Upload File"])
            text_input = ""
            with tab1:
                raw = st.text_area("Enter text (minimum 50 characters):", height=220, key="ra_text")
                if raw:
                    text_input = raw
            with tab2:
                st.markdown('''<style>[data-testid="stFileUploadDropzone"]{background-color:#ffffff !important;border:2px solid #1a1a1a !important;border-radius:10px !important;}[data-testid="stFileUploadDropzone"] span,[data-testid="stFileUploadDropzone"] p,[data-testid="stFileUploadDropzone"] small{color:#1a1a1a !important;}[data-testid="stFileUploadDropzone"] button{background:#1a1a1a !important;border:none !important;color:#ffffff !important;border-radius:6px !important;}[data-testid="stFileUploaderFile"] span,[data-testid="stFileUploaderFile"] p,[data-testid="stFileUploaderFile"] a,[data-testid="stFileUploaderFile"] small{color:#1a1a1a !important;text-decoration:none !important;}</style>''', unsafe_allow_html=True)
                uploaded = st.file_uploader("Upload a TXT or PDF file", type=["txt", "pdf"], key="ra_file")
                if uploaded:
                    try:
                        if uploaded.type == "application/pdf":
                            reader     = PyPDF2.PdfReader(uploaded)
                            text_input = "\n".join(p.extract_text() for p in reader.pages if p.extract_text())
                            st.info(f"✅ Loaded {len(reader.pages)} page(s) from PDF.")
                        else:
                            text_input = uploaded.read().decode("utf-8")
                            st.info(f"✅ Loaded: {uploaded.name}")
                    except Exception as e:
                        st.error(f"Error reading file: {e}")
            sp("sp16")
            if st.button("Analyze →", key="ra_btn"):
                if len(text_input.strip()) < 50:
                    st.error("Text too short — please enter at least 50 characters.")
                else:
                    with st.spinner("Calculating readability metrics…"):
                        analyzer = rl.ReadabilityAnalyzer(text_input)
                        scores   = analyzer.get_all_metrics()
                    st.markdown("---")
                    st.markdown("<h3 style='color:#1a1a1a; font-family:Playfair Display,serif;'>📊 Results</h3>", unsafe_allow_html=True)
                    avg_grade = (scores["Flesch-Kincaid Grade"] + scores["Gunning Fog"] +
                                 scores["SMOG Index"] + scores["Coleman-Liau"]) / 4
                    if avg_grade <= 6:    level, color = "Beginner (Elementary)",          "#27ae60"
                    elif avg_grade <= 10: level, color = "Intermediate (Middle School)",   "#2980b9"
                    elif avg_grade <= 14: level, color = "Advanced (High School/College)", "#f39c12"
                    else:                 level, color = "Expert (Professional/Academic)", "#e74c3c"
                    st.markdown(f"""
                    <div class="level-badge" style="background:{color}18; border-left:5px solid {color};">
                      <span style="color:{color};">{level}</span>
                      <span style="font-size:14px; color:#9a948b; font-family:'DM Sans'; font-weight:300; margin-left:12px;">≈ Grade {int(avg_grade)}</span>
                    </div>
                    """, unsafe_allow_html=True)
                    st.markdown("<h4 style='color:#1a1a1a; margin-top:20px;'>📈 Metric Gauges</h4>", unsafe_allow_html=True)
                    c1, c2, c3 = st.columns(3)
                    with c1:
                        st.plotly_chart(create_gauge(scores["Flesch Reading Ease"],  "Flesch Ease",   0, 100, "#1a1a1a"), use_container_width=True)
                        with st.expander("ℹ️ Flesch Reading Ease"):
                            st.markdown("<p style='color:#5a5a5a; font-size:13px;'>0–100 scale. Higher = easier. 60–70 is standard readable text.</p>", unsafe_allow_html=True)
                    with c2:
                        st.plotly_chart(create_gauge(scores["Flesch-Kincaid Grade"], "Kincaid Grade", 0,  20, "#5b4fcf"), use_container_width=True)
                        with st.expander("ℹ️ Flesch-Kincaid Grade"):
                            st.markdown("<p style='color:#5a5a5a; font-size:13px;'>US school grade level. 8 means an 8th grader can understand.</p>", unsafe_allow_html=True)
                    with c3:
                        st.plotly_chart(create_gauge(scores["SMOG Index"],           "SMOG Index",    0,  20, "#e67e22"), use_container_width=True)
                        with st.expander("ℹ️ SMOG Index"):
                            st.markdown("<p style='color:#5a5a5a; font-size:13px;'>Used for medical writing. Based on polysyllabic words.</p>", unsafe_allow_html=True)
                    c4, c5 = st.columns(2)
                    with c4:
                        st.plotly_chart(create_gauge(scores["Gunning Fog"],  "Gunning Fog",  0, 20, "#16a085"), use_container_width=True)
                        with st.expander("ℹ️ Gunning Fog"):
                            st.markdown("<p style='color:#5a5a5a; font-size:13px;'>Based on sentence length and complex words.</p>", unsafe_allow_html=True)
                    with c5:
                        st.plotly_chart(create_gauge(scores["Coleman-Liau"], "Coleman-Liau", 0, 20, "#8e44ad"), use_container_width=True)
                        with st.expander("ℹ️ Coleman-Liau"):
                            st.markdown("<p style='color:#5a5a5a; font-size:13px;'>Uses character counts — good for automated text analysis.</p>", unsafe_allow_html=True)
                    st.markdown("<h4 style='color:#1a1a1a; margin-top:20px;'>📝 Text Statistics</h4>", unsafe_allow_html=True)
                    s1, s2, s3, s4, s5 = st.columns(5)
                    s1.metric("Sentences",     analyzer.num_sentences)
                    s2.metric("Words",         analyzer.num_words)
                    s3.metric("Syllables",     analyzer.num_syllables)
                    s4.metric("Complex Words", analyzer.complex_words)
                    s5.metric("Characters",    analyzer.char_count)

        elif nav == "🛡️  Admin":
            if uemail != ADMIN_EMAIL:
                st.error("⛔ Admin access only.")
            else:
                st.markdown("<h3 style='color:#1a1a1a;'>🛡️ Admin Panel</h3>", unsafe_allow_html=True)
                users = db.get_all_users()
                st.metric("Total Registered Users", len(users))
                st.markdown("---")
                col1, col2, col3, col4 = st.columns([1, 3, 3, 1])
                col1.markdown("**ID**"); col2.markdown("**Username / Email**")
                col3.markdown("**Joined**"); col4.markdown("**Action**")
                st.markdown("---")
                for uid, uname_, ueml, sec_q, created in users:
                    c1, c2, c3, c4 = st.columns([1, 3, 3, 1])
                    c1.write(uid)
                    c2.write(f"**{uname_}**  \n{ueml}")
                    c3.write(created or "—")
                    if ueml != ADMIN_EMAIL:
                        if c4.button("Delete", key=f"del_{uid}"):
                            db.delete_user(ueml)
                            st.warning(f"Deleted {ueml}")
                            st.rerun()


if __name__ == "__main__":
    main()

Overwriting app.py


In [27]:
import os
from pyngrok import ngrok

for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

os.system("pkill -9 streamlit")
os.system("pkill -9 ngrok")
ngrok.kill()
print("✅ Killed.")

✅ Killed.


In [28]:
import time
time.sleep(10)
print("✅ Ready.")

✅ Ready.


In [29]:
from pyngrok import ngrok
from google.colab import userdata
import subprocess, time, os

NGROK_TOKEN = userdata.get("NGROK_AUTHTOKEN")
ngrok.set_auth_token(NGROK_TOKEN)

env = os.environ.copy()
env["EMAIL_ID"]           = userdata.get("EMAIL_ID")
env["EMAIL_APP_PASSWORD"] = userdata.get("EMAIL_APP_PASSWORD")
env["JWT_SECRET_KEY"]     = userdata.get("JWT_SECRET_KEY")
env["ADMIN_EMAIL_ID"]     = userdata.get("ADMIN_EMAIL_ID")

process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    env=env
)

time.sleep(8)
public_url = ngrok.connect(8501)
print("🌍 Live at:", public_url)

🌍 Live at: NgrokTunnel: "https://chubby-vania-unwonderfully.ngrok-free.dev" -> "http://localhost:8501"


In [30]:

import sqlite3
import pandas as pd

conn = sqlite3.connect("users.db")
print("=== USERS ===")
print(pd.read_sql_query("SELECT id, username, email, security_q, created_at FROM users", conn))
print("\n=== PASSWORD HISTORY ===")
print(pd.read_sql_query("SELECT email, set_at FROM password_history ORDER BY set_at DESC LIMIT 20", conn))
print("\n=== LOGIN ATTEMPTS ===")
print(pd.read_sql_query("SELECT * FROM login_attempts", conn))
conn.close()

=== USERS ===
   id username                        email              security_q  \
0   1    ramya  kuruvalaramya2006@gmail.com  What is your pet name?   

            created_at  
0  2026-02-22 18:39:21  

=== PASSWORD HISTORY ===
                         email               set_at
0  kuruvalaramya2006@gmail.com  2026-02-22 18:52:39
1  kuruvalaramya2006@gmail.com  2026-02-22 18:39:21

=== LOGIN ATTEMPTS ===
Empty DataFrame
Columns: [email, attempts, last_attempt]
Index: []
